In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_digits
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [2]:
# Load dataset (Digits dataset)
digits = load_digits()
X = digits.data
y = digits.target

In [3]:
# Preprocessing
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [4]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [5]:
# Function to build a simple neural network model
def create_model(learning_rate=0.01, num_neurons=32):
    model = Sequential([
        Dense(num_neurons, input_dim=X_train.shape[1], activation='relu'),
        Dense(10, activation='softmax')
    ])
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [6]:
model_before_tuning = create_model()
history_before = model_before_tuning.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_split=0.2)
accuracy_before = model_before_tuning.evaluate(X_test, y_test, verbose=0)[1]

print(f"Accuracy before hyperparameter tuning: {accuracy_before * 100:.2f}%")

C:\Desktop\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Accuracy before hyperparameter tuning: 97.78%


In [7]:
learning_rates = [0.01, 0.001]
batch_sizes = [16, 32]
num_neurons_list = [32, 64]

In [8]:
# Hyperparameter tuning using grid search
def grid_search(X_train, y_train, X_test, y_test):
    # Initialize best_accuracy to a very low value
    best_accuracy = -float('inf')
    best_params = {}

    for lr in learning_rates:
        for batch_size in batch_sizes:
            for num_neurons in num_neurons_list:
                model = create_model(learning_rate=lr, num_neurons=num_neurons)
                history = model.fit(X_train, y_train, epochs=20, batch_size=batch_size, verbose=0, validation_split=0.2)
                
                # Evaluate on test set
                accuracy = model.evaluate(X_test, y_test, verbose=0)[1]
                
                # Update best accuracy and parameters
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params = {'learning_rate': lr, 'batch_size': batch_size, 'num_neurons': num_neurons}
    
    return best_params, best_accuracy

# Call grid_search and pass test data
best_params, accuracy_after = grid_search(X_train, y_train, X_test, y_test)

print(f"Best hyperparameters: {best_params}")
print(f"Accuracy after hyperparameter tuning: {accuracy_after * 100:.2f}%")


Best hyperparameters: {'learning_rate': 0.01, 'batch_size': 32, 'num_neurons': 32}
Accuracy after hyperparameter tuning: 98.33%
